In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkSession available as 'spark'.


In [43]:
import pickle
phrase2frq = pickle.load(open("/Users/lakerwayne/Desktop/phrase2frq.pickle",'rb'))
print phrase2frq

{'sliced_jalapenos': 0, 'pasteurized_egg': 0, 'cucumber_soup': 0, 'roasted_tomatoes': 0, 'herb_infused': 0, 'gorgonzola_cream': 0, 'dishes_tasted': 0, 'tartar_sauce': 0, 'grated_parm': 0, 'added_flavor': 0, 'cheese_curds': 0, 'falafel_balls': 0, 'pomme_frites': 0, 'parm_cheese': 0, 'beef_enchilada': 0, 'sliced_meats': 0, 'gorgonzola_pizza': 0, 'extremely_spicy': 0, 'petite_fours': 0, 'hue_style': 0, 'chashu_pork': 0, 'fried_gyoza': 0, 'duck_pizza': 0, 'blue_salad': 0, 'beef_patty': 0, 'spicy_salsas': 0, 'chocolate_chips': 0, 'rice_vermicelli': 0, 'souvlaki_dinners': 0, 'pra_ram': 0, 'siracha_mayo': 0, 'black_pepper': 0, 'cilantro_jalapeno': 0, 'ama_ebi': 0, 'barely_touched': 0, 'duck_salad': 0, 'cajun_chicken': 0, 'kids_loved': 0, 'shawarma_plates': 0, 'wonton_tacos': 0, 'cinnamon_sticks': 0, 'fried_dishes': 0, 'chicken_fundido': 0, 'hubs_got': 0, 'idaho_potatoes': 0, 'souvlaki_platter': 0, 'mole_verde': 0, 'sweet_tangy': 0, 'cooked_med': 0, 'schnitzel_sandwich': 0, 'pulled_pork': 0, '

In [3]:
import os
os.chdir('/Users/lakerwayne/Desktop/YelpChallenge')
from process import Restaurant
rev_pickle = pickle.load(open('/Users/lakerwayne/Desktop/YelpChallenge/data_edinburg_rest.pickle', 'rb'))

In [28]:
rid2reviews = []
for city in rev_pickle:
    for i,r in enumerate(rev_pickle[city]):
        reviews = ""
        for c in rev_pickle[city][r].texts:
            reviews += c
        rev = tuple([rev_pickle[city][r].rid, reviews])
        rid2reviews.append(rev)

In [29]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf

sentenceDataFrame = spark.createDataFrame(rid2reviews, ["rid", "review"])
tokenizer = Tokenizer(inputCol="review", outputCol="words")
tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.show(5)

+--------------------+--------------------+--------------------+
|                 rid|              review|               words|
+--------------------+--------------------+--------------------+
|Nvi9RLcOdrJvSwcw-...|The idea of this ...|[the, idea, of, t...|
|XGMI18dOgrOo_r7Wz...|My wife and I ate...|[my, wife, and, i...|
|0HyuO5rOKeaa08TFz...|This is such a go...|[this, is, such, ...|
|HYDfO3jMTHDywO64A...|From the outside ...|[from, the, outsi...|
|paxRLjJUE2IQ-WyNq...|I'd say this is y...|[i'd, say, this, ...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [30]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
tokenized = remover.transform(tokenized)

In [31]:
from pyspark.sql.functions import struct
from pyspark.sql.types import *
import re

def cleanup_text(record):
    text = record[3]
    text_out = [re.sub('[^a-zA-Z0-9]','',word) for word in text]
    return text_out

# define udf with an array of tokenized words
udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
clean_text = tokenized.withColumn("results", udf_cleantext(struct([tokenized[x] for x in tokenized.columns])))

In [32]:
clean_text.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 rid|              review|               words|            filtered|             results|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Nvi9RLcOdrJvSwcw-...|The idea of this ...|[the, idea, of, t...|[idea, pub, extre...|[idea, pub, extre...|
|XGMI18dOgrOo_r7Wz...|My wife and I ate...|[my, wife, and, i...|[wife, ate, twice...|[wife, ate, twice...|
|0HyuO5rOKeaa08TFz...|This is such a go...|[this, is, such, ...|[gorgeous, pub,, ...|[gorgeous, pub, c...|
|HYDfO3jMTHDywO64A...|From the outside ...|[from, the, outsi...|[outside, looks, ...|[outside, looks, ...|
|paxRLjJUE2IQ-WyNq...|I'd say this is y...|[i'd, say, this, ...|[i'd, say, best, ...|[id, say, best, b...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [33]:
domain = clean_text.select('rid','results')

In [34]:
results = domain.select('results').collect()
rids = domain.select('rid').collect()

In [57]:
rid2frq = {}
for i, row in enumerate(results):
    restid = rids[i][0]
    rid2frq[restid] = {}
    word_list = row[0]
    for widx, w in enumerate(word_list):
        if widx==len(word_list)-1:
                continue
        dish = w+"_"+word_list[widx+1]
        if dish in phrase2frq:
            if dish not in rid2frq[restid]:
                rid2frq[restid][dish] = 0
            rid2frq[restid][dish] += 1
            phrase2frq[dish] += 1

In [59]:
with open("restaurant_frquency.pickle","wb") as p:
    pickle.dump(rid2frq,p)

In [49]:
count = 0
while(count < 20):
    for p in sorted(phrase2frq, key=phrase2frq.get, reverse=True):
        print p + ": " + str(phrase2frq[p])
        count += 1

ice_cream: 693
pretty_good: 601
fish_chips: 475
main_course: 463
gluten_free: 403
sweet_potato: 281
hot_chocolate: 277
black_pudding: 273
smoked_salmon: 253
lunch_menu: 252
main_courses: 216
tasted_like: 214
perfectly_cooked: 213
choose_from: 209
absolutely_delicious: 201
pulled_pork: 198
nothing_special: 194
pork_belly: 192
baked_potato: 191
deep_fried: 190
goats_cheese: 189
well_cooked: 185
nice_touch: 172
high_quality: 170
chicken_wings: 159
home_made: 157
eggs_benedict: 156
flat_white: 154
spring_rolls: 152
red_wine: 149
freshly_made: 148
white_wine: 146
pad_thai: 145
sour_cream: 143
portion_size: 140
la_carte: 140
cant_remember: 135
sea_bass: 135
blue_cheese: 133
cooked_perfectly: 132
well_done: 129
big_fan: 123
everything_else: 122
well_presented: 122
tomato_sauce: 120
vegetarian_options: 119
sticky_toffee: 117
dim_sum: 117
carrot_cake: 115
olive_oil: 113
potato_fries: 111
different_types: 110
miso_soup: 108
toffee_pudding: 107
baked_potatoes: 104
beer_selection: 104
fried_rice: 

perfect_ending: 6
shredded_duck: 6
yaki_soba: 6
ciabatta_bread: 6
biggest_disappointment: 6
hot_green: 6
antipasti_platter: 6
cole_slaw: 6
perfectly_poached: 6
bad_boy: 6
pork_carnitas: 6
sounded_great: 6
short_rib: 6
malt_vinegar: 6
chocolate_truffles: 6
creamed_spinach: 6
beef_tartare: 6
desperately_needed: 6
various_sauces: 6
sad_looking: 6
tall_glass: 6
freshly_fried: 6
mixed_platter: 6
fresh_garlic: 6
spicy_curry: 6
green_chicken: 6
veg_tempura: 6
bleu_cheese: 6
fresh_crab: 6
toasted_almonds: 6
slightly_cold: 6
mint_leaves: 6
jack_cheese: 6
pickled_veg: 6
cabernet_sauvignon: 6
burger_arrived: 6
olive_tapenade: 6
egg_benedict: 6
spinach_dish: 6
half_size: 6
fried_beef: 6
lamb_chop: 6
vanilla_shake: 6
creamy_cheese: 6
italian_pasta: 6
pasta_salad: 6
corn_tortillas: 6
mexican_style: 6
yellow_curry: 6
cheese_naan: 6
el_diablo: 6
interesting_toppings: 6
stainless_steel: 6
burger_buns: 6
cajun_fries: 6
bit_lighter: 6
flourless_chocolate: 6
chocolate_fountain: 6
hot_tea: 6
salty_sweet: 6

cod_taco: 1
extremely_rich: 1
tasty_fries: 1
carne_tacos: 1
ground_lamb: 1
combo_meal: 1
open_flame: 1
fresh_sashimi: 1
golden_bags: 1
crispy_noodles: 1
butter_lettuce: 1
fried_scallops: 1
juicy_burger: 1
tomato_chunks: 1
dry_rubbed: 1
egg_pancake: 1
grilled_cod: 1
rare_meat: 1
lemon_butter: 1
gyro_plate: 1
spinach_gnocchi: 1
prepared_medium: 1
jack_fruit: 1
rosemary_potato: 1
linguini_carbonara: 1
chicken_parma: 1
ny_steak: 1
baby_greens: 1
ordered_sliders: 1
entree_arrived: 1
assorted_sashimi: 1
poutine_gravy: 1
mini_taco: 1
bmb_special: 1
pork_jowl: 1
indian_hot: 1
flying_fish: 1
popcorn_shrimp: 1
canned_tomato: 1
tuscan_chicken: 1
duck_hash: 1
hot_bar: 1
tossed_salad: 1
cooked_foods: 1
parmesan_pork: 1
pasta_tasted: 1
cold_rolls: 1
evaporated_milk: 1
pasta_bolognese: 1
risotto_dish: 1
walnut_bread: 1
pork_inside: 1
crispy_garlic: 1
kids_spaghetti: 1
chicken_shrimp: 1
veg_biryani: 1
egg_pudding: 1
pickled_turnip: 1
lettuce_bun: 1
seafood_tofu: 1
onion_pizza: 1
fruit_bowl: 1
pho_soup

khao_soy: 0
waygu_beef: 0
apple_brie: 0
entire_pie: 0
add_mushrooms: 0
spoon_pizza: 0
spicy_shoyu: 0
hand_battered: 0
family_recipe: 0
large_horchata: 0
raw_seafood: 0
chunky_salsa: 0
meatball_spaghetti: 0
mole_chicken: 0
blow_torched: 0
cheeseburger_combo: 0
fried_jalapeos: 0
cucumber_yogurt: 0
corn_shell: 0
torta_bread: 0
rolled_taquitos: 0
veal_tenderloin: 0
sized_meatballs: 0
pata_negra: 0
loaded_potato: 0
pork_stomach: 0
chicken_karage: 0
veggie_broth: 0
pancit_bihon: 0
brick_chicken: 0
phoenix_roll: 0
pastry_basket: 0
afghan_rice: 0
tomatillo_salsa: 0
jalapeo_mayo: 0
fried_shallots: 0
alla_nora: 0
korean_kalbi: 0
nigerian_prawn: 0
jerked_chicken: 0
chili_verde: 0
tuna_bowl: 0
peppered_bacon: 0
bo_ssam: 0
chicken_based: 0
filet_minion: 0
antibiotic_free: 0
farm_burger: 0
german_chocolate: 0
creme_brulees: 0
white_radish: 0
light_brown: 0
veal_picatta: 0
caramelized_cauliflower: 0
elk_sliders: 0
az_burger: 0
fried_raviolis: 0
pollo_guisado: 0
garlic_edamame: 0
fried_nicely: 0
add_i

enchiladas_verdes: 0
hoison_sauce: 0
cinnamon_gelato: 0
hara_bhara: 0
pei_mussels: 0
rad_nah: 0
basa_fish: 0
beef_shoulder: 0
roasted_eggplant: 0
rad_nar: 0
steamed_vegetable: 0
yari_ika: 0
spicy_vinegar: 0
taiwanese_style: 0
shio_broth: 0
jalapeo_peppers: 0
beef_vermicelli: 0
salt_rim: 0
stuffed_zucchini: 0
kobe_style: 0
appetizer_sampler: 0
blush_sauce: 0
shrimp_roll: 0
whip_cream: 0
chick_parm: 0
curry_ketchup: 0
merlot_butter: 0
potato_harra: 0
katsu_donburi: 0
vegan_sausage: 0
italian_picnic: 0
ethiopian_coffee: 0
specialty_tacos: 0
italian_hoagie: 0
jalapeo_yellowtail: 0
sour_orange: 0
sea_urchin: 0
duck_buns: 0
spring_dosa: 0
roasted_tomatillo: 0
coffee_gelato: 0
favorite_pasta: 0
crispy_corn: 0
chiu_chow: 0
nice_snap: 0
egg_plant: 0
souvlaki_plate: 0
beef_kabobs: 0
three_sauces: 0
strawberry_sauce: 0
jamon_iberico: 0
sesame_flavour: 0
agua_de: 0
lemon_zest: 0
camarones_culichi: 0
spicy_mint: 0
hai_nam: 0
entrees_arrived: 0
picante_pizza: 0
lobster_benedict: 0
asparagus_roll: 0


In [51]:
with open("phrase_count.txt","w") as txtfile:
    for p in sorted(phrase2frq, key=phrase2frq.get, reverse=True):
        txtfile.write(p + ": " + str(phrase2frq[p]))
        txtfile.write("\n")